In [ ]:
import pandas as pd
import numpy as np

import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

from google.cloud import bigquery

In [ ]:
transactions = pd.read_csv('transactions.csv')
buildings = pd.read_csv('building.csv')
schools = pd.read_csv('schools.csv')
medianrent = pd.read_csv('rent.csv')

In [ ]:
transactions.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [ ]:
schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [ ]:
medianrent

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-
...,...,...,...,...
11272,2023-Q1,YISHUN,2-RM,-
11273,2023-Q1,YISHUN,3-RM,2500
11274,2023-Q1,YISHUN,4-RM,3000
11275,2023-Q1,YISHUN,5-RM,3200


In [ ]:
buildings.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0


In [ ]:
medianrent['flat_type'].unique()

array(['1-RM', '2-RM', '3-RM', '4-RM', '5-RM', 'EXEC'], dtype=object)

In [ ]:
transactions['flat_type'].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

In [ ]:
def load_flat_type(transactions):
    flat_type = pd.DataFrame(transactions['flat_type'].unique(), columns = ['flat_type'])
    return flat_type

load_flat_type(transactions)

,flat_type
0,2 ROOM
1,3 ROOM
2,4 ROOM
3,5 ROOM
4,EXECUTIVE
5,1 ROOM
6,MULTI-GENERATION


In [ ]:
def make_title(df):
    string_columns = df.select_dtypes(include=['object'])
    df[string_columns.columns] = string_columns.applymap(str.title)
    return df

make_title(transactions)

/var/folders/s8/h576kn8n48vd37_dt88pwm0w0000gn/T/ipykernel_13804/2939670584.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[string_columns.columns] = string_columns.applymap(str.title)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,Ang Mo Kio,2 Room,406,Ang Mo Kio Ave 10,10 To 12,44.0,Improved,1979,61 Years 04 Months,232000.0
1,2017-01,Ang Mo Kio,3 Room,108,Ang Mo Kio Ave 4,01 To 03,67.0,New Generation,1978,60 Years 07 Months,250000.0
2,2017-01,Ang Mo Kio,3 Room,602,Ang Mo Kio Ave 5,01 To 03,67.0,New Generation,1980,62 Years 05 Months,262000.0
3,2017-01,Ang Mo Kio,3 Room,465,Ang Mo Kio Ave 10,04 To 06,68.0,New Generation,1980,62 Years 01 Month,265000.0
4,2017-01,Ang Mo Kio,3 Room,601,Ang Mo Kio Ave 5,01 To 03,67.0,New Generation,1980,62 Years 05 Months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
175722,2024-03,Yishun,Executive,392,Yishun Ave 6,07 To 09,146.0,Maisonette,1988,63 Years 04 Months,830000.0
175723,2024-03,Yishun,Executive,406,Yishun Ave 6,10 To 12,142.0,Apartment,1988,63 Years 06 Months,815000.0
175724,2024-03,Yishun,Executive,326,Yishun Ring Rd,10 To 12,146.0,Maisonette,1988,63 Years 05 Months,820000.0
175725,2024-03,Yishun,Executive,360,Yishun Ring Rd,04 To 06,146.0,Maisonette,1988,63 Years 05 Months,860000.0


In [ ]:
buildings

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,33,50,0,0,0,0,0,0,0,0
12822,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,...,31,47,0,0,0,0,0,0,0,0
12823,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,...,0,0,0,0,0,0,0,0,0,0
12824,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,0,0,0,0,0,0,136,170,0,0


In [ ]:
def onemap_api(data):
    async def fetch(session, blk_no, street):
        search_val = f"{blk_no} {street}"
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

        async with session.get(url) as response:
            try:
                query = await response.json()
                query = query['results'][0]  
                postal = query['POSTAL']
                return postal
            except:
                print('API Query failed')
                return None
            
    async def main():
        tasks = []
        async with aiohttp.ClientSession() as session:
            for index, row in data.iterrows():
                task = asyncio.create_task(fetch(session, row['blk_no'], row['street']))
                tasks.append(task)
            results = await asyncio.gather(*tasks)
        return results

    return asyncio.run(main())

In [ ]:
postal_codes = onemap_api(buildings)

In [ ]:
buildings

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,33,50,0,0,0,0,0,0,0,0
12822,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,...,31,47,0,0,0,0,0,0,0,0
12823,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,...,0,0,0,0,0,0,0,0,0,0
12824,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,0,0,0,0,0,0,136,170,0,0


In [ ]:
def transform_buildings(buildings):
    retained_cols = ['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'multistorey_carpark', 'precinct_pavilion', 'market_hawker']
    buildings = buildings[retained_cols]

    # Run OneMap API to get postal code
    buildings['postal_code'] = onemap_api(buildings)
    return buildings

#transform_buildings(buildings)